In [ ]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from webdriver_manager.chrome import ChromeDriverManager
# import mysql.connector
# import re
# import time

# # MySQL 연결 설정
# connection = mysql.connector.connect(
#     host='localhost',
#     user='ohgiraffers',
#     password='ohgiraffers',
#     database='test_db1',
#     charset='utf8mb4'
# )

# # Selenium 웹 드라이버 설정
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# base_url = 'https://www.bysuco.com/product?num=60&page='
# page_number = 1  # 페이지 번호 시작

# while True:
#     # 각 페이지에 접근
#     driver.get(f'{base_url}{page_number}&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')
#     time.sleep(2)

#     # 상품 목록에서 상세 페이지 링크 추출
#     product_links = []
#     elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
    
#     # 상품이 없으면 루프 종료
#     if not elements:
#         print("더 이상 상품이 없습니다. 수집을 종료합니다.")
#         break

#     for element in elements:
#         href = element.get_attribute('href')
#         product_links.append(href)

#     # 각 상품 상세 페이지에서 텍스트 및 이미지 URL 추출
#     for link in product_links:
#         driver.get(link)
#         time.sleep(2)

#         try:
#             # 페이지가 완전히 로드될 때까지 기다림
#             WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap')))

#             # 필요한 데이터 추출
#             text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
#             name = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
#             brand_name = driver.find_element(By.CLASS_NAME, 'tit').text
#             desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text
            
#             # 이미지 처리 개선
#             image_urls = []
            
#             # swiper-wrapper 내 load 클래스의 img src 속성 추출
#             swiper_wrapper = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CLASS_NAME, 'swiper-slide-active'))
#             )
#             load_images = swiper_wrapper.find_elements(By.CLASS_NAME, 'load')

#             # load 클래스의 img 요소 중 src 속성을 추출
#             for img in load_images:
#                 image_url = img.get_attribute('src')
#                 print(f"이미지 URL: {image_url}")

#             # 텍스트에서 각 항목 추출
#             grade = re.search(r"\[부향률\]\s*-\s*([^\n]+)", desc_wrap)
#             grade = grade.group(1).strip() if grade else "Unknown"
            
#             top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
#             top_note = top_note.group(1).strip() if top_note else "Unknown"

#             middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
#             middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

#             base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
#             base_note = base_note.group(1).strip() if base_note else "Unknown"
            
#             single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
#             single_note = single_note.group(1).strip() if single_note else "Unknown"

#             description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
#             description = description.group(1).strip() if description else "Unknown"

#             # MySQL 데이터 삽입
#             with connection.cursor() as cursor:
#                 # 테이블 생성
#                 cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     name VARCHAR(50),
#                     brand VARCHAR(50),
#                     grade VARCHAR(50),
#                     description TEXT
#                 )''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS perfume_image (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     img_url TEXT,
#                     perfume_id BIGINT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS top_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     top_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS middle_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     middle_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS base_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     base_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS single_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     single_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')

#                 try:
#                     # perfume 테이블에 데이터 삽입
#                     cursor.execute("""
#                         INSERT INTO perfume (name, brand, description, grade) 
#                         VALUES (%s, %s, %s, %s)
#                     """, (name, brand_name, description, grade))
#                     perfume_id = cursor.lastrowid
                    
#                     # 이미지 URL을 image 테이블에 삽입
#                     cursor.execute("INSERT INTO perfume_image (img_url, perfume_id) VALUES (%s,%s)", (image_url, perfume_id))
                    
#                     # 각 노트 삽입
#                     if top_note != "Unknown":
#                         cursor.execute("INSERT INTO top_note (perfume_id, top_note) VALUES (%s, %s)", 
#                                     (perfume_id, top_note))
#                     if middle_note != "Unknown":
#                         cursor.execute("INSERT INTO middle_note (perfume_id, middle_note) VALUES (%s, %s)", 
#                                     (perfume_id, middle_note))
#                     if base_note != "Unknown":
#                         cursor.execute("INSERT INTO base_note (perfume_id, base_note) VALUES (%s, %s)", 
#                                     (perfume_id, base_note))
#                     if single_note != "Unknown":
#                         cursor.execute("INSERT INTO single_note (perfume_id, single_note) VALUES (%s, %s)", 
#                                     (perfume_id, single_note))
                    
#                     # 모든 쿼리가 성공적으로 실행되면 커밋
#                     connection.commit()

#                 except Exception as e:
#                     # 오류 발생 시 롤백
#                     connection.rollback()
#                     print(f"데이터 삽입 중 오류 발생: {e}")
#                     continue

#         except Exception as e:
#             print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")
#             continue  # 에러가 발생하면 다음 상품으로 넘어감

#     # 다음 페이지로 이동
#     page_number += 1

# # 드라이버 종료 및 데이터베이스 연결 종료
# driver.quit()
# connection.close()

# print("모든 데이터가 성공적으로 저장되었습니다!")

In [32]:
# 향수 디비로 만들기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import mysql.connector
import re
import time

# MySQL 연결 설정
connection = mysql.connector.connect(
    host='localhost',
    user='ohgiraffers',
    password='ohgiraffers',
    database='test_db',
    charset='utf8mb4'
)

# Selenium 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)  # 명시적 대기 시간 설정
base_url = 'https://www.bysuco.com/product?num=60&page='
page_number = 1

while True:
    try:
        # 각 페이지에 접근
        driver.get(f'{base_url}{page_number}&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')
        time.sleep(2)  # 페이지 로딩 대기 시간 증가

        # 상품 목록에서 상세 페이지 링크 추출
        product_links = []
        elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="/product/show"]')))
        
        if not elements:
            print("더 이상 상품이 없습니다. 수집을 종료합니다.")
            break

        # 링크 수집
        for element in elements:
            href = element.get_attribute('href')
            if href not in product_links:
                product_links.append(href)

        # 각 상품 상세 페이지 처리
        for link in product_links:
            try:
                driver.get(link)
                time.sleep(2)  # 페이지 로딩 대기

                # 기본 정보 추출
                try:
                    name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'desc.ellipsisTwo'))).text
                    print(f"상품명: {name}")  # 디버깅용 출력
                    
                    brand_name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'tit'))).text
                    print(f"브랜드: {brand_name}")  # 디버깅용 출력
                    
                    desc_wrap = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap'))).text
                    print("설명 텍스트 추출 완료")  # 디버깅용 출력
                except TimeoutException as e:
                    print(f"기본 정보 추출 중 타임아웃: {e}")
                    continue

                # 이미지 처리
                image_urls = []
                try:
                    # 먼저 현재 표시된 이미지의 URL 가져오기
                    main_image = wait.until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, '.swiper-slide-active img'))
                    )
                    main_url = main_image.get_attribute('src')
                    if main_url:
                        image_urls.append(main_url)
                        print(f"메인 이미지 URL 추가: {main_url}")

                    # 모든 이미지 슬라이드 찾기
                    all_slides = driver.find_elements(By.CSS_SELECTOR, '.swiperWrap.mb .swiper-slide')
                    print(f"발견된 총 슬라이드 수: {len(all_slides)}")

                    # 각 슬라이드 순회
                    for i in range(len(all_slides)):
                        try:
                            # 현재 슬라이드의 이미지 찾기
                            current_slide = all_slides[i]
                            img = current_slide.find_element(By.TAG_NAME, 'img')
                            image_url = img.get_attribute('src')
                            
                            if image_url and image_url not in image_urls:
                                image_urls.append(image_url)
                                print(f"이미지 URL {i+1} 추가: {image_url}")

                        except Exception as e:
                            print(f"슬라이드 {i+1} 처리 중 오류: {e}")
                            continue

                except Exception as e:
                    print(f"이미지 처리 중 오류 발생: {e}")

                print(f"총 수집된 이미지 URL 수: {len(image_urls)}")

                # 텍스트 정보 추출
                grade = re.search(r"\[부향률\]\s*-\s*([^\n]+)", desc_wrap)
                grade = grade.group(1).strip() if grade else "Unknown"
                
                top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
                top_note = top_note.group(1).strip() if top_note else "Unknown"

                middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
                middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

                base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
                base_note = base_note.group(1).strip() if base_note else "Unknown"
                
                single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
                single_note = single_note.group(1).strip() if single_note else "Unknown"

                description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
                description = description.group(1).strip() if description else "Unknown"

                # MySQL 데이터 삽입
                with connection.cursor() as cursor:
                    # 테이블 생성
                    cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        name VARCHAR(50),
                        brand VARCHAR(50),
                        grade VARCHAR(50),
                        description TEXT
                    )''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS perfume_image (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        img_url TEXT,
                        perfume_id BIGINT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS top_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        top_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS middle_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        middle_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS base_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        base_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS single_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        single_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')

                    try:
                        # perfume 테이블에 데이터 삽입
                        cursor.execute("""
                            INSERT INTO perfume (name, brand, description, grade) 
                            VALUES (%s, %s, %s, %s)
                        """, (name, brand_name, description, grade))
                        perfume_id = cursor.lastrowid
                        
                        # 이미지 URL 저장
                        for url in image_urls:
                            cursor.execute("""
                                INSERT INTO perfume_image (img_url, perfume_id) 
                                VALUES (%s, %s)
                            """, (url, perfume_id))
                        
                        # 노트 정보 저장
                        if top_note != "Unknown":
                            cursor.execute("INSERT INTO top_note (perfume_id, top_note) VALUES (%s, %s)", 
                                        (perfume_id, top_note))
                        if middle_note != "Unknown":
                            cursor.execute("INSERT INTO middle_note (perfume_id, middle_note) VALUES (%s, %s)", 
                                        (perfume_id, middle_note))
                        if base_note != "Unknown":
                            cursor.execute("INSERT INTO base_note (perfume_id, base_note) VALUES (%s, %s)", 
                                        (perfume_id, base_note))
                        if single_note != "Unknown":
                            cursor.execute("INSERT INTO single_note (perfume_id, single_note) VALUES (%s, %s)", 
                                        (perfume_id, single_note))
                        
                        connection.commit()
                        print(f"상품 '{name}' 데이터 저장 완료")

                    except Exception as e:
                        connection.rollback()
                        print(f"데이터베이스 저장 중 오류: {e}")
                        continue

            except Exception as e:
                print(f"상품 페이지 처리 중 오류 발생: {e}")
                continue

        page_number += 1

    except Exception as e:
        print(f"페이지 처리 중 오류 발생: {e}")
        break

driver.quit()
connection.close()
print("크롤링이 완료되었습니다!")

상품명: 오드 우드 오 드 퍼퓸
브랜드: 톰 포드
설명 텍스트 추출 완료
메인 이미지 URL 추가: https://cf.bysuco.net/b26c9b393726d01fd533ef5cabcc632f_cvt.webp?w=600
발견된 총 슬라이드 수: 0
총 수집된 이미지 URL 수: 1
상품 '오드 우드 오 드 퍼퓸' 데이터 저장 완료
상품명: 오르페옹 오 드 퍼퓸
브랜드: 딥티크
설명 텍스트 추출 완료
메인 이미지 URL 추가: https://cf.bysuco.net/4b722975836a2e17226bfc5af864e36a_cvt.webp?w=600
발견된 총 슬라이드 수: 0
총 수집된 이미지 URL 수: 1
상품 '오르페옹 오 드 퍼퓸' 데이터 저장 완료
상품명: 플레르 드 뽀 오 드 퍼퓸
브랜드: 딥티크
설명 텍스트 추출 완료
메인 이미지 URL 추가: https://cf.bysuco.net/674f1342fa588659762bd7e538e87297_cvt.webp?w=600
발견된 총 슬라이드 수: 0
총 수집된 이미지 URL 수: 1
상품 '플레르 드 뽀 오 드 퍼퓸' 데이터 저장 완료
상품명: 휠 오 드 퍼퓸
브랜드: 이솝
설명 텍스트 추출 완료
메인 이미지 URL 추가: https://cf.bysuco.net/e0ca52e42b76ec856d659e552e564dd9.jpg?w=600
발견된 총 슬라이드 수: 0
총 수집된 이미지 URL 수: 1
상품 '휠 오 드 퍼퓸' 데이터 저장 완료
상품명: 테싯 오 드 퍼퓸
브랜드: 이솝
설명 텍스트 추출 완료
메인 이미지 URL 추가: https://cf.bysuco.net/fb3db1f776b96fd2bd74015405501147.jpg?w=600
발견된 총 슬라이드 수: 0
총 수집된 이미지 URL 수: 1
상품 '테싯 오 드 퍼퓸' 데이터 저장 완료
상품명: 어벤투스 오 드 퍼퓸
브랜드: 크리드
설명 텍스트 추출 완료
메인 이미지 URL 추가: https://cf.bysuco.net/25aa369

In [41]:
# 향료 db만들기(image_id 매핑)
import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import Error
from urllib.parse import urljoin  # urljoin을 사용하여 상대경로를 절대경로로 변환

# DB 연결
try:
    connection = mysql.connector.connect(
        host='localhost',
        user='ohgiraffers',
        password='ohgiraffers',
        database='test_db',
        charset='utf8mb4',
        use_unicode=True
    )
    
    cursor = connection.cursor(buffered=True)

    # 테이블 생성
    cursor.execute(''' 
        CREATE TABLE IF NOT EXISTS line (
            id BIGINT PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(255),
            content VARCHAR(255),
            color VARCHAR(50)
        )
    ''')

    cursor.execute(''' 
        CREATE TABLE IF NOT EXISTS spice (
            id BIGINT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(255) NOT NULL,
            content TEXT,
            line_id BIGINT,
            FOREIGN KEY (line_id) REFERENCES line(id) ON DELETE CASCADE
        )
    ''')

    cursor.execute(''' 
        CREATE TABLE IF NOT EXISTS spice_image (
            id BIGINT AUTO_INCREMENT PRIMARY KEY,
            img_url TEXT NOT NULL,
            spice_id BIGINT,
            FOREIGN KEY (spice_id) REFERENCES spice(id) ON DELETE CASCADE
        )
    ''')

    # line 데이터 삽입 (이미 삽입된 데이터가 있을 경우 무시할 수 있도록 처리)
    cursor.execute(''' 
        INSERT IGNORE INTO line (name, content, color) VALUES
            ('Spicy', '자극적이며 섹시한 향', 'FF5757'),
            ('Cypre', '격조있고 성숙한 느낌의 개성있는 향', 'FF7F43'),
            ('Fruity', '달콤하고 귀여운 향', 'FFBD43'),
            ('Citrus', '과일 같은 상큼한 향', 'FFE043'),
            ('Green', '상쾌하고 신선한 향', '62D66A'),
            ('Aldehyde', '인공 비누향, 모던하고 우아하면 지적인 향', '98D1FF'),
            ('Aquatic', '들과 바다 시원함이 가득한 향', '56D2FF'),
            ('Fougere', '부드럽고 감미로우며 편안한 향', 'FFD9A6'),
            ('Gourmand', '달달하고 유혹적인 미각의 즐거움을 느낄 수 있는 향', 'A1522C'),
            ('Woody', '고상하고 증후한 느낌의 따뜻하고 부드러운 향', '86390F'),
            ('Oriental', '무거우면서 부드럽고 고혹적인 향, 증후하고 달콤하며 센슈얼한 향', 'C061FF'),
            ('Floral', '화려하고 여성스러운 향', 'FF80C1'),
            ('Musk', '이성에게 어필 가능한 아름다운 향', 'F8E4FF'),
            ('Powdery', '포근한 향', 'FFFFFF'),
            ('Amber', '달콤하고 부드러운 향', 'FFA75A'),
            ('Tobacco Leather', '마초적인 남성의 개성이 강한 매력적인 향', '000000')
    ''')

    # 향료 데이터를 가져올 URL 설정
    url = "https://www.fragrantica.com/notes/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    notes = soup.find_all('div', class_='cell small-6 medium-4 large-3 text-center notebox')

    if not notes:
        print("향료 데이터를 찾지 못했습니다.")
    else:
        for note in notes:
            try:
                a_tag = note.find('a')
                if not a_tag:
                    continue
                
                name = a_tag.text.strip()
                note_url = urljoin("https://www.fragrantica.com", a_tag.get('href'))  # urljoin 사용
                img_tag = note.find('img')
                image_url = img_tag.get('src') if img_tag else None
                
                # 향료 상세 페이지로 요청
                detail_response = requests.get(note_url, headers=headers)
                detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

                # 'cell callout' 클래스를 찾아서 내용 추출
                callout_tag = detail_soup.find('div', class_='cell callout')  # 클래스 이름 수정
                content = None
                if callout_tag:
                    p_tag = callout_tag.find('p')  # p 태그를 찾음
                    if p_tag:
                        content = p_tag.get_text(strip=True)  # p 태그의 텍스트 추출
                    else:
                        content = None  # p 태그가 없는 경우

                if not content:
                    print(f"컨텐츠 없음 - Name: {name}")
                    continue

                # spice 데이터 삽입
                cursor.execute(
                    "INSERT INTO spice (name, content, line_id) VALUES (%s, %s, %s)",
                    (name, content, 1)  # line_id는 1로 설정 (적절히 변경 가능)
                )
                spice_id = cursor.lastrowid  # 삽입된 spice의 id 가져오기

                # 이미지 데이터 삽입
                if image_url:
                    cursor.execute("INSERT INTO spice_image (img_url, spice_id) VALUES (%s, %s)", (image_url, spice_id))
                print(f"데이터 삽입 성공 - Name: {name}, Content: {content}, Image URL: {image_url}")
                
                connection.commit()  # 각 항목마다 커밋
            except Exception as e:
                print(f"항목 처리 중 오류 발생: {e}")
                connection.rollback()
                continue

        print("모든 데이터가 성공적으로 저장되었습니다!")

except Exception as e:
    print(f"프로그램 실행 중 오류 발생: {e}")
    if connection:
        connection.rollback()

finally:
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 연결이 종료되었습니다.")

향료 데이터를 찾지 못했습니다.
MySQL 연결이 종료되었습니다.
